In [ ]:
import pandas as pd
import ast  # For safely evaluating string-formatted lists
import plotly.express as px

arenas_dict = {  
       54000002:      "Arena 1",                      
       54000003:      "Arena 2",                  
       54000004:      "Arena 3",                  
       54000005:      "Arena 4",                  
       54000006:      "Arena 5",                  
       54000008:      "Arena 6",                  
       54000009:      "Arena 7",                  
       54000010:      "Arena 8",                  
       54000007:      "Arena 9",                  
       54000024:      "Arena 10",            
       54000011:      "Arena 11",                  
       54000055:      "Arena 12",                  
       54000056:      "Arena 13",                  
       54000012:      "Arena 14",                  
       54000013:      "Arena 15",                  
       54000014:      "Arena 16",                  
       54000015:      "Arena 17",                  
       54000016:      "Arena 18",                  
       54000017:      "Arena 19",                  
       54000018:      "Arena 20",                  
       54000019:      "Arena 21",                  
       54000020:      "Arena 22",                  
       54000031:      "Arena 23",                
       54000117:      "Arena 24",     
}

# --- Helper Function ---
def check_for_card(card_list_str, card_name):
    """
    Safely parses a string representation of a card list
    and checks if a specific card is present.
    """
    try:
        # ast.literal_eval is safer than eval()
        card_list = ast.literal_eval(card_list_str)
        if not isinstance(card_list, list):
            return False
        
        # The list contains tuples like ('Card Name', level, 0)
        for card_tuple in card_list:
            if isinstance(card_tuple, tuple) and len(card_tuple) > 0 and card_tuple[0] == card_name:
                return True
    except (ValueError, SyntaxError, TypeError):
        # Handles cases where the string is not a valid list (e.g., NaN, malformed)
        return False
    return False

def plot_arena_card_win_loss(card_name):
    try:
        # 1. Load the dataset
        file_name = '../#2 Data Storage/Processed Data/preprocessed_battle_log.csv'
        df = pd.read_csv(file_name)
        CARD_TO_FIND = card_name

        print(f"Data loaded. Processing for '{CARD_TO_FIND}' ...")

        # 2. Initialize list to store results
        card_data = []

        # 3. Iterate through all battles
        for row in df.itertuples():
            # Check player 0
            player_0_has_card = check_for_card(row.players_0_spells, CARD_TO_FIND)
            if player_0_has_card:
                card_data.append({
                    'arena': row.arena,
                    'outcome': 'Won' if row.players_0_winner == 1 else 'Lost'
                })
            
            # Check player 1
            player_1_has_card = check_for_card(row.players_1_spells, CARD_TO_FIND)
            if player_1_has_card:
                card_data.append({
                    'arena': row.arena,
                    'outcome': 'Won' if row.players_1_winner == 1 else 'Lost'
                })

        # Create a new DataFrame with Mega Knight usage data
        if not card_data:
            print(f"No usage data found for '{CARD_TO_FIND}'.")
        else:
            card_df = pd.DataFrame(card_data)
            
            # Group data by arena and outcome to get counts
            grouped_df = card_df.groupby(['arena', 'outcome']).size().reset_index(name='count')
            grouped_df['arena'] = grouped_df['arena']
            grouped_df['arena'] = grouped_df['arena'].map(arenas_dict)  # Map arena IDs to names
            grouped_df = grouped_df.dropna(subset=['arena'])

            arena_order = sorted(list(set(grouped_df['arena'])), key=lambda x: int(x.split(' ')[1]))
            grouped_df['arena'] = pd.Categorical(grouped_df['arena'], categories=arena_order, ordered=True)
            grouped_df = grouped_df.sort_values(by='arena')

            print(f"\nAggregated {CARD_TO_FIND} Usage Data:")
            print(grouped_df)

            color_map = {
            'Won': 'green',
            'Lost': 'red'
            }
            
            # Create the visualization using Plotly Express
            fig = px.bar(
                grouped_df,
                x='arena',
                y='count',
                color='outcome',
                title=f'{CARD_TO_FIND} Usage Count per Arena by Win/Loss',
                labels={
                    'arena': 'Arena',
                    'count': f'{CARD_TO_FIND} Usage Count',
                    'outcome': f'Outcome for User'
                },
                barmode='group',
                color_discrete_map=color_map
            )
            
            fig.update_layout(xaxis=dict(type='category'))  # Ensure x-axis treats arenas as categories
            fig.show()
            
    except FileNotFoundError:
        print(f"Error: The file '{file_name}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [15]:
plot_arena_card_win_loss('Valkyrie')

Data loaded. Processing for 'Valkyrie' ...

Aggregated Valkyrie Usage Data:
       arena outcome  count
0    Arena 1    Lost     28
1    Arena 1     Won     21
2    Arena 2    Lost    133
3    Arena 2     Won    120
4    Arena 3    Lost    210
5    Arena 3     Won    210
6    Arena 4    Lost    357
7    Arena 4     Won    374
8    Arena 5    Lost    294
9    Arena 5     Won    342
12   Arena 6    Lost    331
13   Arena 6     Won    367
14   Arena 7    Lost    303
15   Arena 7     Won    326
17   Arena 8     Won    321
16   Arena 8    Lost    302
11   Arena 9     Won    255
10   Arena 9    Lost    259
41  Arena 10     Won    388
40  Arena 10    Lost    356
18  Arena 11    Lost    362
19  Arena 11     Won    386
56  Arena 12     Won    329
55  Arena 12    Lost    320
58  Arena 13     Won    434
57  Arena 13    Lost    374
20  Arena 14    Lost    354
21  Arena 14     Won    392
22  Arena 15    Lost    156
23  Arena 15     Won    160
24  Arena 16    Lost    207
25  Arena 16     Won    220
